In [100]:
import numpy as np
from heat_equation import temp_result, Temp_list_analit_parallel
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [101]:
def plot_temp(temp_data, length=1, t_max=100):
    N_t, N_x = temp_data.shape
    x = np.linspace(0, length, N_x)
    t = np.linspace(0, t_max, N_t)
    X, Y = np.meshgrid(x, t)
    
    fig = go.Figure()
    fig.add_trace(go.Surface(z=temp_data, x=X, y=Y))

    fig.update_layout(
        title="3D График температуры",
        width=1000,
        height=800,
        scene=dict(
            xaxis_title="x",
            yaxis_title="time",
            zaxis_title="Temperature",
            zaxis=dict(range=[0, np.max(temp_data)])  
        ),
    )
    fig.show()

def compute_error(result, exact_solution, t):
    error = 0
    for x in range(1, len(result[t])-1):
        error_ = abs((result[t][x] - exact_solution[t][x]) / exact_solution[t][x])
        if error < error_:
            error = error_
    return error

def compute_s(result, t_n):
    s = 0
    for x in range(len(result[t_n])):
        s_ = abs((result[t_n][x] - result[t_n - 1][x]) / result[t_n][x])
        if s < s_:
            s = s_
    return s


# Аналитиеское решение

In [102]:
length = 20
t_max = 200
a = 1
u_0 = 100
n_x = 100
n_t = 500
terms = 100

In [103]:
Temp_analit = Temp_list_analit_parallel(n_x, n_t, length, t_max, a, u_0, terms)
# plot_temp(Temp_analit, length, t_max)

# Численное решение

In [104]:
# Euler_method == 0, Nicholson_Crank_method == 1, Nicholson_Crank_method_modified == 2

length = 20     #40
t_max = 200
a = 1
u_0 = 100
n_x = 90
n_t = 8000       #2500

initial_conditions = np.full(n_x, u_0, dtype=np.float64)
initial_conditions[0] = 0
initial_conditions[-1] = 0

## Метод Эйлера

In [105]:
result_euler = np.zeros((n_t, n_x), dtype=np.float64)
temp_result(0, initial_conditions, result_euler, 
            a, n_x, n_t,
            length, t_max)

In [106]:
# plot_temp(result_euler, length, t_max)

## Метод Кранка-Николсона

In [107]:
result_crank = np.zeros((n_t, n_x), dtype=np.float64)
temp_result(1, initial_conditions, result_crank,
                        a, n_x, n_t,
                        length, t_max)

In [108]:
# plot_temp(result_crank, length, t_max)

## Начальное условие sin(pi * x / L)

In [109]:
# Начальное условие sin(pi * x / L)

initial_conditions_sin = [np.sin(np.pi * x / length) for x in np.linspace(0, length, n_x)]

In [110]:
result_euler_sin = np.zeros((n_t, n_x), dtype=np.float64)
result_crank_sin = np.zeros((n_t, n_x), dtype=np.float64)


temp_result(0, initial_conditions_sin, result_euler_sin,
            a, n_x, n_t,
            length, t_max)

temp_result(1, initial_conditions_sin, result_crank_sin,
            a, n_x, n_t,
            length, t_max)


In [111]:
# plot_temp(result_euler_sin, length, t_max)

In [112]:
# plot_temp(result_crank_sin, length, t_max)

# Построение ошибок

In [113]:
def error_plot(solver_number, t_max, n_x, n_t):
    all_solvers = ['Euler_method', 'Nicholson_Crank_method']
    length = 20     #40
    a = 1
    u_0 = 100

    initial_conditions = np.full(n_x, u_0, dtype=np.float64)
    initial_conditions[0] = 0
    initial_conditions[-1] = 0

    time = np.linspace(0, t_max, n_t)
    error_mas = []

    result = np.zeros((n_t, n_x), dtype=np.float64)
    temp_result(solver_number, initial_conditions, result, 
                a, n_x, n_t,
                length, t_max)
    
    Temp_analit = Temp_list_analit_parallel(n_x, n_t, length, t_max, a, u_0, terms)
    
    for t in range(n_t):
        error_mas.append(compute_error(result, Temp_analit, t))

    plt.figure(figsize=(10, 6)) 
    plt.plot(time, error_mas, label=all_solvers[solver_number])
    plt.xlim(0.01, t_max)  # Ограничиваем ось Y


    
    plt.xlabel('Time')
    plt.ylabel('Error')
    plt.legend()
    plt.grid(True)
    plt.show()

In [114]:
def error_fixed_plot(solver_number, n_x, t_f, e_0):
    all_solvers = ['Euler_method', 'Nicholson_Crank_method']
    length = 20     #40
    a = 1
    u_0 = 100
    n_t = 10

    initial_conditions = np.full(n_x, u_0, dtype=np.float64)
    initial_conditions[0] = 0
    initial_conditions[-1] = 0

    result = np.zeros((n_t, n_x), dtype=np.float64)

    temp_result(solver_number, initial_conditions, result,
                a, n_x, n_t,
                length, t_f)
    

    Temp_analit = Temp_list_analit_parallel(n_x, n_t, length, t_f, a, u_0, terms)

    while(compute_error(result, Temp_analit, n_t-1) > e_0):
            n_t += 10
            result = np.zeros((n_t, n_x), dtype=np.float64)
            temp_result(solver_number, initial_conditions, result,
                        a, n_x, n_t,
                        length, t_f)
            Temp_analit = Temp_list_analit_parallel(n_x, n_t, length, t_f, a, u_0, terms)
    
    return n_t



In [115]:
def s_plot(solver_number, t_max, n_x, n_t, s_0 = 0.01):
    all_solvers = ['Euler_method', 'Nicholson_Crank_method']
    length = 20     #40
    a = 1
    u_0 = 100


    initial_conditions = np.full(n_x, u_0, dtype=np.float64)
    initial_conditions[0] = 0
    initial_conditions[-1] = 0

    time = np.linspace(0, t_max, n_t-1)
    s_mas = []
    s_0_mas = []

    result = np.zeros((n_t, n_x), dtype=np.float64)
    temp_result(solver_number, initial_conditions, result,
                a, n_x, n_t,
                length, t_max)
    
    for t in range(1, n_t):
        s_mas.append(compute_s(result, t))
        s_0_mas.append(s_0)

    plt.figure(figsize=(10, 6)) 
    plt.plot(time, s_mas, label=all_solvers[solver_number])
    plt.plot(time, s_0_mas, label='S_0')
    plt.ylim(s_0 / 2, s_0 * 3 / 2)  # Ограничиваем ось Y

    plt.xlabel('time')
    plt.ylabel('S')
    plt.legend()
    plt.grid(True)
    plt.show()

In [116]:
def dichotomy(x_array, y_array, y0, tol=1e-6, max_iter=100):
    left = 0
    right = len(x_array) - 1
    
    for _ in range(max_iter):
        mid = (left + right) // 2
        y_mid = y_array[mid]
        
        if abs(y_mid - y0) < tol:
            return mid
        
        if (y_array[left] - y0) * (y_mid - y0) > 0:
            left = mid
        else:
            right = mid
    return mid

In [117]:
t_max = 50
n_x = 50
n_t = 900
# error_plot(1, t_max, n_x, n_t)

In [118]:
t_max = 10
n_x = 100
n_t = 500
# s_plot(0, t_max, n_x, n_t)

In [119]:
t_max = 50
n_x = 100
n_t = 2500
s_0 = 0.0005
# s_plot(1, t_max, n_x, n_t, s_0)

In [120]:
# Зададим S0 и найдем момент времени, когда такая степень стационарности достигается

all_solvers = ['Euler_method', 'Nicholson_Crunk_method']
length = 20     #40

initial_conditions = np.full(n_x, u_0, dtype=np.float64)
initial_conditions[0] = 0
initial_conditions[-1] = 0

time = np.linspace(0, t_max, n_t-1)
s_mas = []
s_0_mas = []

result = np.zeros((n_t, n_x), dtype=np.float64)
temp_result(1, initial_conditions, result,
            a, n_x, n_t,
            length, t_max)

for t in range(1, n_t):
    s_mas.append(compute_s(result, t))

time = np.linspace(0, t_max, n_t)
t_f = time[dichotomy(time, s_mas, s_0)]
print(f"t_f = {t_f}")



t_f = 32.79311724689876


/tmp/ipykernel_55340/3627036580.py:34: RuntimeWarning:

invalid value encountered in scalar divide



In [121]:
# Зададим E0 
e_0 = 1e-3
n_t_optimal = error_fixed_plot(1, n_x, t_f, e_0)
print(n_t_optimal)


340


# Задание 5

In [122]:
all_solvers = ['Euler_method', 'Nicholson_Crunk_method']
length = 20     #40
t_max = 100
a = 0.9
u_0 = 100
u_1 = 50
n_x = 100
n_t = 8500       #2500

initial_conditions = []
for i in range(n_x):
    if i < n_x / 2:
        initial_conditions.append(u_0)
    else:
        initial_conditions.append(u_1)

initial_conditions[0] = 0
initial_conditions[-1] = 0

In [123]:
result = np.zeros((n_t, n_x), dtype=np.float64)
temp_result(1, initial_conditions, result,
            a, n_x, n_t,
            length, t_max)

# plot_temp(result, length, t_max)

In [124]:
# Задача Ньютона

all_solvers = ['Euler_method', 'Nicholson_Crunk_method', 'Nicholson_Crunk_method_modified']
length = 20     #40
t_max = 200
a = 0.5
u_0 = 100

h = 0.1
u_c = 20

n_x = 90
n_t = 4000       #2500

initial_conditions = np.full(n_x, u_0, dtype=np.float64)
initial_conditions[0] = 0
initial_conditions[-1] = 0

In [125]:
# 1 стержень

result = np.zeros((n_t, n_x), dtype=np.float64)
result_crank = temp_result(2, initial_conditions, result, 
                        a, n_x, n_t,
                        length, t_max,
                        h, u_c)
# plot_temp(result, length, t_max)

In [126]:
initial_conditions = []
for i in range(n_x):
    if i < n_x / 2:
        initial_conditions.append(u_0)
    else:
        initial_conditions.append(u_1)

initial_conditions[0] = 0
initial_conditions[-1] = 0


result = np.zeros((n_t, n_x), dtype=np.float64)
result_crank = temp_result(2, initial_conditions, result, 
                        a, n_x, n_t,
                        length, t_max,
                        h, u_c)

# plot_temp(result, length, t_max)